In [9]:
# General imports
import numpy as np
import pandas as pd
import os, sys, gc, time, warnings, pickle, psutil, random

# custom imports
from multiprocessing import Pool        # Multiprocess Runs

warnings.filterwarnings('ignore')

In [10]:
# :seed to make all processes deterministic     # type: int
def seed_everything(seed=0):
    random.seed(seed)
    np.random.seed(seed)

    
## Multiprocess Runs
def df_parallelize_run(func, t_split):
    num_cores = np.min([N_CORES,len(t_split)])
    pool = Pool(num_cores)
    df = pd.concat(pool.map(func, t_split), axis=1)
    pool.close()
    pool.join()
    return df

In [11]:
# Read data
def get_data_by_store(store):
    
    # Read and contact basic feature
    df = pd.concat([pd.read_pickle(BASE),
                    pd.read_pickle(PRICE).iloc[:,2:],
                    pd.read_pickle(CALENDAR).iloc[:,2:]],
                    axis=1)
    
    # Leave only relevant store
    df = df[df['store_id']==store]

    # With memory limits we have to read 
    # lags and mean encoding features
    # separately and drop items that we don't need.
    # As our Features Grids are aligned 
    # we can use index to keep only necessary rows
    # Alignment is good for us as concat uses less memory than merge.
    df2 = pd.read_pickle(MEAN_ENC)[mean_features]
    df2 = df2[df2.index.isin(df.index)]
    
    df3 = pd.read_pickle(LAGS).iloc[:,3:]
    df3 = df3[df3.index.isin(df.index)]
    
    df = pd.concat([df, df2], axis=1)
    del df2 # to not reach memory limit 
    
    df = pd.concat([df, df3], axis=1)
    del df3 # to not reach memory limit 
    
    # Create features list
    features = [col for col in list(df) if col not in remove_features]
    df = df[['id','d',TARGET]+features]
    
    # Skipping first n rows
    df = df[df['d']>=START_TRAIN].reset_index(drop=True)
    
    return df, features

# Recombine Test set after training
def get_base_test():
    base_test = pd.DataFrame()

    for store_id in STORES_IDS:
        temp_df = pd.read_pickle('test_'+store_id+'.pkl')
        temp_df['store_id'] = store_id
        base_test = pd.concat([base_test, temp_df]).reset_index(drop=True)
    
    return base_test


########################### Helper to make dynamic rolling lags
#################################################################################
def make_lag(LAG_DAY):
    lag_df = base_test[['id','d',TARGET]]
    col_name = 'sales_lag_'+str(LAG_DAY)
    lag_df[col_name] = lag_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(LAG_DAY)).astype(np.float16)
    return lag_df[[col_name]]


def make_lag_roll(LAG_DAY):
    shift_day = LAG_DAY[0]
    roll_wind = LAG_DAY[1]
    lag_df = base_test[['id','d',TARGET]]
    col_name = 'rolling_mean_tmp_'+str(shift_day)+'_'+str(roll_wind)
    lag_df[col_name] = lag_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(shift_day).rolling(roll_wind).mean())
    return lag_df[[col_name]]

In [12]:
import lightgbm as lgb
lgb_params = {
                    'boosting_type': 'gbdt',
                    'objective': 'tweedie',
                    'tweedie_variance_power': 1.1,
                    'metric': 'rmse',
                    'subsample': 0.5,
                    'subsample_freq': 1,
                    'learning_rate': 0.03,
                    'num_leaves': 2**11-1,
                    'min_data_in_leaf': 2**12-1,
                    'feature_fraction': 0.5,
                    'max_bin': 100,
                    'n_estimators': 1400,
                    'boost_from_average': False,
                    'verbose': -1,
                } 

# Let's look closer on params

## 'boosting_type': 'gbdt'
# we have 'goss' option for faster training
# but it normally leads to underfit.
# Also there is good 'dart' mode
# but it takes forever to train
# and model performance depends 
# a lot on random factor 
# https://www.kaggle.com/c/home-credit-default-risk/discussion/60921

## 'objective': 'tweedie'
# Tweedie Gradient Boosting for Extremely
# Unbalanced Zero-inflated Data
# https://arxiv.org/pdf/1811.10192.pdf
# and many more articles about tweediie
#
# Strange (for me) but Tweedie is close in results
# to my own ugly loss.
# My advice here - make OWN LOSS function
# https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/140564
# https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/143070
# I think many of you already using it (after poisson kernel appeared) 
# (kagglers are very good with "params" testing and tuning).
# Try to figure out why Tweedie works.
# probably it will show you new features options
# or data transformation (Target transformation?).

## 'tweedie_variance_power': 1.1
# default = 1.5
# set this closer to 2 to shift towards a Gamma distribution
# set this closer to 1 to shift towards a Poisson distribution
# my CV shows 1.1 is optimal 
# but you can make your own choice

## 'metric': 'rmse'
# Doesn't mean anything to us
# as competition metric is different
# and we don't use early stoppings here.
# So rmse serves just for general 
# model performance overview.
# Also we use "fake" validation set
# (as it makes part of the training set)
# so even general rmse score doesn't mean anything))
# https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/133834

## 'subsample': 0.5
# Serves to fight with overfit
# this will randomly select part of data without resampling
# Chosen by CV (my CV can be wrong!)
# Next kernel will be about CV

##'subsample_freq': 1
# frequency for bagging
# default value - seems ok

## 'learning_rate': 0.03
# Chosen by CV
# Smaller - longer training
# but there is an option to stop 
# in "local minimum"
# Bigger - faster training
# but there is a chance to
# not find "global minimum" minimum

## 'num_leaves': 2**11-1
## 'min_data_in_leaf': 2**12-1
# Force model to use more features
# We need it to reduce "recursive"
# error impact.
# Also it leads to overfit
# that's why we use small 

# 'max_bin': 100
## l1, l2 regularizations
# https://towardsdatascience.com/l1-and-l2-regularization-methods-ce25e7fc831c
# Good tiny explanation
# l2 can work with bigger num_leaves
# but my CV doesn't show boost
                    
## 'n_estimators': 1400
# CV shows that there should be
# different values for each state/store.
# Current value was chosen 
# for general purpose.
# As we don't use any early stopings
# careful to not overfit Public LB.

##'feature_fraction': 0.5
# LightGBM will randomly select 
# part of features on each iteration (tree).
# We have maaaany features
# and many of them are "duplicates"
# and many just "noise"
# good values here - 0.5-0.7 (by CV)

## 'boost_from_average': False
# There is some "problem"
# to code boost_from_average for 
# custom loss
# 'True' makes training faster
# BUT carefull use it
# https://github.com/microsoft/LightGBM/issues/1514

In [13]:
VER = 1                          # Our model version
SEED = 42                        # We want all things
seed_everything(SEED)            # to be as deterministic 
lgb_params['seed'] = SEED        # as possible
N_CORES = psutil.cpu_count()     # Available CPU cores


#LIMITS and const
TARGET      = 'sales'            # Our target
START_TRAIN = 0                  # We can skip some rows (Nans/faster training)
END_TRAIN   = 1913               # End day of our train set
P_HORIZON   = 28                 # Prediction horizon

#FEATURES to remove
## These features lead to overfit
## or values not present in test set
remove_features = ['id','state_id','store_id',
                   'date','wm_yr_wk','d',TARGET]
mean_features   = ['enc_cat_id_mean','enc_cat_id_std',
                   'enc_dept_id_mean','enc_dept_id_std',
                   'enc_item_id_mean','enc_item_id_std'] 

#PATHS for Features
BASE     = 'grid_part_1.pkl'
PRICE    = 'grid_part_2.pkl'
CALENDAR = 'grid_part_3.pkl'
LAGS     = 'lags_df_28.pkl'
MEAN_ENC = 'mean_encoding_df.pkl'


# AUX(pretrained) Models paths

#STORES ids
STORES_IDS = pd.read_csv('sales_train_validation.csv')['store_id']
STORES_IDS = list(STORES_IDS.unique())


#SPLITS for lags creation
SHIFT_DAY  = 28
N_LAGS     = 15
LAGS_SPLIT = [col for col in range(SHIFT_DAY,SHIFT_DAY+N_LAGS)]
ROLS_SPLIT = []
for i in [1,7,14]:
    for j in [7,14,28,56]:
        ROLS_SPLIT.append([i,j])

In [6]:
for store_id in STORES_IDS:
    print('Train', store_id)
    
    # Get grid for current store
    grid_df, features_columns = get_data_by_store(store_id)
    
    print(features_columns)
    # Masks for 
    # Train (All data less than 1913)
    # "Validation" (Last 28 days - not real validation set)
    # Test (All data greater than 1913 day, 
    #       with some gap for recursive features)
    train_mask = grid_df['d']<=END_TRAIN
    valid_mask = train_mask&(grid_df['d']>(END_TRAIN-P_HORIZON))
    preds_mask = grid_df['d']>(END_TRAIN-100)
    
    # Apply masks and save lgb dataset as bin
    # to reduce memory spikes during dtype convertations
    # https://github.com/Microsoft/LightGBM/issues/1032
    # "To avoid any conversions, you should always use np.float32"
    # or save to bin before start training
    # https://www.kaggle.com/c/talkingdata-adtracking-fraud-detection/discussion/53773
    train_data = lgb.Dataset(grid_df[train_mask][features_columns], 
                       label=grid_df[train_mask][TARGET])
    train_data.save_binary('train_data.bin')
    train_data = lgb.Dataset('train_data.bin')
    
    valid_data = lgb.Dataset(grid_df[valid_mask][features_columns], 
                       label=grid_df[valid_mask][TARGET])
    X_valid=grid_df[valid_mask][features_columns]
    
    valid_pred = grid_df[valid_mask].reset_index(drop=True)
    cols = [col for col in ['id', 'd']]
    valid_pred = valid_pred[cols]
    
    # Saving part of the dataset for later predictions
    # Removing features that we need to calculate recursively 
    grid_df = grid_df[preds_mask].reset_index(drop=True)
    keep_cols = [col for col in list(grid_df) if '_tmp_' not in col]
    grid_df = grid_df[keep_cols]
    grid_df.to_pickle('test_'+store_id+'.pkl')
    del grid_df
    gc.collect()
    
    # Launch seeder again to make lgb training 100% deterministic
    # with each "code line" np.random "evolves" 
    # so we need (may want) to "reset" it
    seed_everything(SEED)
    estimator = lgb.train(lgb_params,
                          train_data,
                          valid_sets = [valid_data],
                          verbose_eval = 100,
                          )
    
    valid_pred['pred']=estimator.predict(X_valid)
    valid_pred.to_pickle('valid_pred_'+store_id+'.pkl')
    del valid_pred
    gc.collect()
    
    # Save model - it's not real '.bin' but a pickle file
    # estimator = lgb.Booster(model_file='model.txt')
    # can only predict with the best iteration (or the saving iteration)
    # pickle.dump gives us more flexibility
    # like estimator.predict(TEST, num_iteration=100)
    # num_iteration - number of iteration want to predict with, 
    # NULL or <= 0 means use best iteration
    model_name = 'lgb_model_'+store_id+'_v'+str(VER)+'.bin'
    pickle.dump(estimator, open(model_name, 'wb'))

    # Remove temporary files and objects 
    # to free some hdd space and ram memory
    !rm train_data.bin
    del train_data, valid_data, estimator
    gc.collect()
    
    # "Keep" models features for predictions
    MODEL_FEATURES = features_columns

Train CA_1
['item_id', 'dept_id', 'cat_id', 'release', 'sell_price', 'price_max', 'price_min', 'price_std', 'price_mean', 'price_norm', 'price_rank_dept', 'price_nunique', 'item_nunique', 'price_momentum', 'price_momentum_m', 'price_momentum_y', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI', 'is_first_half_month', 'event_bef_weekend', 'event_after_weekend', 'NBA', 'event_attention_after', 'event_attention_bef', 'event_attention_sum', 'tm_d', 'tm_w', 'tm_m', 'tm_q', 'tm_y', 'tm_wm', 'tm_dw', 'tm_w_end', 'enc_cat_id_mean', 'enc_cat_id_std', 'enc_dept_id_mean', 'enc_dept_id_std', 'enc_item_id_mean', 'enc_item_id_std', 'sales_lag_28', 'sales_lag_29', 'sales_lag_30', 'sales_lag_31', 'sales_lag_32', 'sales_lag_33', 'sales_lag_34', 'sales_lag_35', 'sales_lag_36', 'sales_lag_37', 'sales_lag_38', 'sales_lag_39', 'sales_lag_40', 'sales_lag_41', 'sales_lag_42', 'rolling_mean_7', 'rolling_std_7', 'rolling_mean_14', 'rolling_std_14', 'rolling_mean_2

[100]	valid_0's rmse: 1.6085
[200]	valid_0's rmse: 1.58783
[300]	valid_0's rmse: 1.57979
[400]	valid_0's rmse: 1.57404
[500]	valid_0's rmse: 1.5685
[600]	valid_0's rmse: 1.5645
[700]	valid_0's rmse: 1.56027
[800]	valid_0's rmse: 1.55623
[900]	valid_0's rmse: 1.55215
[1000]	valid_0's rmse: 1.54888
[1100]	valid_0's rmse: 1.54501
[1200]	valid_0's rmse: 1.54094
[1300]	valid_0's rmse: 1.53718
[1400]	valid_0's rmse: 1.53382
Train TX_2
['item_id', 'dept_id', 'cat_id', 'release', 'sell_price', 'price_max', 'price_min', 'price_std', 'price_mean', 'price_norm', 'price_rank_dept', 'price_nunique', 'item_nunique', 'price_momentum', 'price_momentum_m', 'price_momentum_y', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI', 'is_first_half_month', 'event_bef_weekend', 'event_after_weekend', 'NBA', 'event_attention_after', 'event_attention_bef', 'event_attention_sum', 'tm_d', 'tm_w', 'tm_m', 'tm_q', 'tm_y', 'tm_wm', 'tm_dw', 'tm_w_end', 'enc_cat_id_mean', '

[100]	valid_0's rmse: 1.92393
[200]	valid_0's rmse: 1.85423
[300]	valid_0's rmse: 1.83104
[400]	valid_0's rmse: 1.81813
[500]	valid_0's rmse: 1.80752
[600]	valid_0's rmse: 1.79835
[700]	valid_0's rmse: 1.79222
[800]	valid_0's rmse: 1.78483
[900]	valid_0's rmse: 1.7799
[1000]	valid_0's rmse: 1.77383
[1100]	valid_0's rmse: 1.76766
[1200]	valid_0's rmse: 1.76205
[1300]	valid_0's rmse: 1.75823
[1400]	valid_0's rmse: 1.75139


In [14]:
# Create Dummy DataFrame to store predictions
all_preds = pd.DataFrame()

# Join back the Test dataset with 
# a small part of the training data 
# to make recursive features
base_test = get_base_test()

# Timer to measure predictions time 
main_time = time.time()

# Loop over each prediction day
# As rolling lags are the most timeconsuming
# we will calculate it for whole day
for PREDICT_DAY in range(1,29):    
    print('Predict | Day:', PREDICT_DAY)
    start_time = time.time()

    # Make temporary grid to calculate rolling lags
    grid_df = base_test.copy()
    grid_df = pd.concat([grid_df, df_parallelize_run(make_lag_roll, ROLS_SPLIT)], axis=1)
        
    for store_id in STORES_IDS:
        
        # Read all our models and make predictions
        # for each day/store pairs
        model_path = 'lgb_model_'+store_id+'_v'+str(VER)+'.bin' 
        
        estimator = pickle.load(open(model_path, 'rb'))
        
        day_mask = base_test['d']==(END_TRAIN+PREDICT_DAY)
        store_mask = base_test['store_id']==store_id
        
        mask = (day_mask)&(store_mask)
        base_test[TARGET][mask] = estimator.predict(grid_df[mask][MODEL_FEATURES])
    
    # Make good column naming and add 
    # to all_preds DataFrame
    temp_df = base_test[day_mask][['id',TARGET]]
    temp_df.columns = ['id','F'+str(PREDICT_DAY)]
    if 'id' in list(all_preds):
        all_preds = all_preds.merge(temp_df, on=['id'], how='left')
    else:
        all_preds = temp_df.copy()
        
    print('#'*10, ' %0.2f min round |' % ((time.time() - start_time) / 60),
                  ' %0.2f min total |' % ((time.time() - main_time) / 60),
                  ' %0.2f day sales |' % (temp_df['F'+str(PREDICT_DAY)].sum()))
    del temp_df
    
all_preds = all_preds.reset_index(drop=True)
all_preds.head()

Predict | Day: 1
##########  0.72 min round |  0.72 min total |  37520.19 day sales |
Predict | Day: 2
##########  0.77 min round |  1.49 min total |  35383.87 day sales |
Predict | Day: 3
##########  0.87 min round |  2.36 min total |  35147.32 day sales |
Predict | Day: 4
##########  0.88 min round |  3.24 min total |  35588.62 day sales |
Predict | Day: 5
##########  0.89 min round |  4.14 min total |  41854.21 day sales |
Predict | Day: 6
##########  0.80 min round |  4.94 min total |  50699.57 day sales |
Predict | Day: 7
##########  0.81 min round |  5.74 min total |  53076.21 day sales |
Predict | Day: 8
##########  0.80 min round |  6.54 min total |  44307.07 day sales |
Predict | Day: 9
##########  0.85 min round |  7.39 min total |  44147.27 day sales |
Predict | Day: 10
##########  0.78 min round |  8.17 min total |  38811.05 day sales |
Predict | Day: 11
##########  0.80 min round |  8.97 min total |  41282.87 day sales |
Predict | Day: 12
##########  0.84 min round |  9.80

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0.841310,0.780816,0.755726,0.732071,0.906987,0.980494,1.100806,0.880358,0.840556,...,0.839435,1.090193,1.028382,0.838720,0.801584,0.742520,0.793294,0.857438,1.054456,0.994806
1,HOBBIES_1_002_CA_1_validation,0.206519,0.191507,0.172134,0.202713,0.218178,0.290731,0.302245,0.209692,0.200329,...,0.205168,0.262049,0.266381,0.182875,0.165946,0.176802,0.187321,0.187024,0.245337,0.268075
2,HOBBIES_1_003_CA_1_validation,0.443345,0.408751,0.429146,0.425363,0.552568,0.638499,0.699324,0.528112,0.471524,...,0.544495,0.735714,0.738077,0.465108,0.409136,0.410397,0.441521,0.555661,0.675365,0.644728
3,HOBBIES_1_004_CA_1_validation,1.672548,1.365069,1.323705,1.507489,2.113503,2.898886,3.210592,1.803483,1.554177,...,1.998574,2.659329,3.237022,1.700170,1.401813,1.410481,1.327478,1.995211,2.862200,3.520612
4,HOBBIES_1_005_CA_1_validation,0.970362,0.842148,0.894868,0.860903,1.058545,1.361341,1.455360,1.033733,1.039133,...,1.119777,1.565799,1.686430,0.982957,0.893096,0.921336,0.922060,1.179444,1.540157,1.550996


In [15]:
# Reading competition sample submission and
# merging our predictions
# As we have predictions only for "_validation" data
# we need to do fillna() for "_evaluation" items
submission = pd.read_csv('sample_submission.csv')[['id']]
submission = submission.merge(all_preds, on=['id'], how='left').fillna(0)
submission.to_csv('lgb_bystore_morefe.csv', index=False) #0.46838

In [ ]:
#calculate wrmsse of valid dataset

In [16]:
ca_1_pred=pd.read_pickle('valid_pred_CA_1.pkl')
print(ca_1_pred.shape)
ca_2_pred=pd.read_pickle('valid_pred_CA_2.pkl')
print(ca_2_pred.shape)
ca_3_pred=pd.read_pickle('valid_pred_CA_3.pkl')
print(ca_3_pred.shape)
ca_4_pred=pd.read_pickle('valid_pred_CA_4.pkl')
print(ca_4_pred.shape)
tx_1_pred=pd.read_pickle('valid_pred_TX_1.pkl')
print(tx_1_pred.shape)
tx_2_pred=pd.read_pickle('valid_pred_TX_2.pkl')
print(tx_2_pred.shape)
tx_3_pred=pd.read_pickle('valid_pred_TX_3.pkl')
print(tx_3_pred.shape)
wi_1_pred=pd.read_pickle('valid_pred_WI_1.pkl')
print(wi_1_pred.shape)
wi_2_pred=pd.read_pickle('valid_pred_WI_2.pkl')
print(wi_2_pred.shape)
wi_3_pred=pd.read_pickle('valid_pred_WI_3.pkl')
print(wi_3_pred.shape)

(85372, 3)
(85372, 3)
(85372, 3)
(85372, 3)
(85372, 3)
(85372, 3)
(85372, 3)
(85372, 3)
(85372, 3)
(85372, 3)


In [17]:
ca_1_pred.head()

,id,d,pred
0,HOBBIES_1_001_CA_1_validation,1886,0.887254
1,HOBBIES_1_002_CA_1_validation,1886,0.175799
2,HOBBIES_1_003_CA_1_validation,1886,0.306288
3,HOBBIES_1_004_CA_1_validation,1886,1.699758
4,HOBBIES_1_005_CA_1_validation,1886,0.965155


In [18]:
valid_pred=pd.concat([ca_1_pred,ca_2_pred,ca_3_pred,ca_4_pred,tx_1_pred,tx_2_pred,tx_3_pred,
                      wi_1_pred,wi_2_pred,wi_3_pred], axis=0).reset_index(drop=True)

In [19]:
valid_pred.shape

(853720, 3)

In [20]:
valid_pred.head()

,id,d,pred
0,HOBBIES_1_001_CA_1_validation,1886,0.887254
1,HOBBIES_1_002_CA_1_validation,1886,0.175799
2,HOBBIES_1_003_CA_1_validation,1886,0.306288
3,HOBBIES_1_004_CA_1_validation,1886,1.699758
4,HOBBIES_1_005_CA_1_validation,1886,0.965155


In [21]:
grid_df=pd.read_pickle('df_full.pkl')
grid_df.columns=['id',
 'item_id',
 'dept_id',
 'cat_id',
 'store_id',
 'state_id',
 'd',
 'sales',
 'release',
 'sell_price',
 'price_max',
 'price_min',
 'price_std',
 'price_mean',
 'price_norm',
 'price_rank_dept',
 'price_nunique',
 'item_nunique',
 'price_momentum',
 'price_momentum_m',
 'price_momentum_y',
 'event_name_1',
 'event_type_1',
 'event_name_2',
 'event_type_2',
 'snap_CA',
 'snap_TX',
 'snap_WI',
 'is_first_half_month',
 'event_bef_weekend',
 'event_after_weekend',
 'NBA',
 'event_attention_after',
 'event_attention_bef',
 'event_attention_sum',
 'tm_d',
 'tm_w',
 'tm_m',
 'tm_q',
 'tm_y',
 'tm_wm',
 'tm_dw',
 'tm_w_end',
 'id_1',
 'd_1',
 'enc_state_id_mean',
 'enc_state_id_std',
 'enc_store_id_mean',
 'enc_store_id_std',
 'enc_cat_id_mean',
 'enc_cat_id_std',
 'enc_dept_id_mean',
 'enc_dept_id_std',
 'enc_state_id_cat_id_mean',
 'enc_state_id_cat_id_std',
 'enc_state_id_dept_id_mean',
 'enc_state_id_dept_id_std',
 'enc_store_id_cat_id_mean',
 'enc_store_id_cat_id_std',
 'enc_store_id_dept_id_mean',
 'enc_store_id_dept_id_std',
 'enc_item_id_mean',
 'enc_item_id_std',
 'enc_item_id_state_id_mean',
 'enc_item_id_state_id_std',
 'enc_item_id_store_id_mean',
 'enc_item_id_store_id_std',
 'sales_lag_28',
 'sales_lag_29',
 'sales_lag_30',
 'sales_lag_31',
 'sales_lag_32',
 'sales_lag_33',
 'sales_lag_34',
 'sales_lag_35',
 'sales_lag_36',
 'sales_lag_37',
 'sales_lag_38',
 'sales_lag_39',
 'sales_lag_40',
 'sales_lag_41',
 'sales_lag_42',
 'rolling_mean_7',
 'rolling_std_7',
 'rolling_mean_14',
 'rolling_std_14',
 'rolling_mean_28',
 'rolling_std_28',
 'rolling_mean_56',
 'rolling_std_56',
 'rolling_mean_168',
 'rolling_std_168',
 'rolling_mean_tmp_1_7',
 'rolling_mean_tmp_1_14',
 'rolling_mean_tmp_1_28',
 'rolling_mean_tmp_1_56',
 'rolling_mean_tmp_7_7',
 'rolling_mean_tmp_7_14',
 'rolling_mean_tmp_7_28',
 'rolling_mean_tmp_7_56',
 'rolling_mean_tmp_14_7',
 'rolling_mean_tmp_14_14',
 'rolling_mean_tmp_14_28',
 'rolling_mean_tmp_14_56']
grid_df.drop(grid_df.columns[43:45], axis=1,inplace=True)

In [22]:
train_mask = grid_df['d']<=END_TRAIN
valid_mask = train_mask&(grid_df['d']>(END_TRAIN-P_HORIZON))

In [ ]:
#valid true sale value

In [23]:
valid=grid_df[valid_mask].reset_index(drop=True)
valid=valid[['id','d', 'sales']]

In [24]:
del grid_df
gc.collect()

95

In [25]:
valid.shape

(853720, 3)

In [26]:
valid.head()

,id,d,sales
0,HOBBIES_1_001_CA_1_validation,1886,1.0
1,HOBBIES_1_002_CA_1_validation,1886,1.0
2,HOBBIES_1_003_CA_1_validation,1886,0.0
3,HOBBIES_1_004_CA_1_validation,1886,0.0
4,HOBBIES_1_005_CA_1_validation,1886,1.0


In [27]:
valid_pred.shape

(853720, 3)

In [28]:
valid_pred.head()

,id,d,pred
0,HOBBIES_1_001_CA_1_validation,1886,0.887254
1,HOBBIES_1_002_CA_1_validation,1886,0.175799
2,HOBBIES_1_003_CA_1_validation,1886,0.306288
3,HOBBIES_1_004_CA_1_validation,1886,1.699758
4,HOBBIES_1_005_CA_1_validation,1886,0.965155


In [29]:
valid_full=pd.merge(valid, valid_pred, how='left', on=['id', 'd'])

In [30]:
valid_full.head()

,id,d,sales,pred
0,HOBBIES_1_001_CA_1_validation,1886,1.0,0.887254
1,HOBBIES_1_002_CA_1_validation,1886,1.0,0.175799
2,HOBBIES_1_003_CA_1_validation,1886,0.0,0.306288
3,HOBBIES_1_004_CA_1_validation,1886,0.0,1.699758
4,HOBBIES_1_005_CA_1_validation,1886,1.0,0.965155


In [31]:
valid_full.tail()

,id,d,sales,pred
853715,FOODS_3_823_WI_3_validation,1913,1.0,0.483800
853716,FOODS_3_824_WI_3_validation,1913,0.0,0.408416
853717,FOODS_3_825_WI_3_validation,1913,0.0,0.784040
853718,FOODS_3_826_WI_3_validation,1913,3.0,1.325899
853719,FOODS_3_827_WI_3_validation,1913,0.0,0.451658


In [32]:
valid = valid.pivot(index="id", columns="d", values="sales").reset_index()

In [33]:
valid.head()

d,id,1886,1887,1888,1889,1890,1891,1892,1893,1894,...,1904,1905,1906,1907,1908,1909,1910,1911,1912,1913
0,FOODS_1_001_CA_1_validation,2.0,1.0,1.0,0.0,4.0,0.0,0.0,4.0,1.0,...,0.0,2.0,0.0,4.0,1.0,1.0,0.0,1.0,1.0,0.0
1,FOODS_1_001_CA_2_validation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,14.0,0.0,1.0,1.0,4.0,0.0,0.0,4.0
2,FOODS_1_001_CA_3_validation,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,13.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,FOODS_1_001_CA_4_validation,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,1.0,1.0
4,FOODS_1_001_TX_1_validation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [34]:
valid.tail()

d,id,1886,1887,1888,1889,1890,1891,1892,1893,1894,...,1904,1905,1906,1907,1908,1909,1910,1911,1912,1913
30485,HOUSEHOLD_2_516_TX_2_validation,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
30486,HOUSEHOLD_2_516_TX_3_validation,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30487,HOUSEHOLD_2_516_WI_1_validation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30488,HOUSEHOLD_2_516_WI_2_validation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
30489,HOUSEHOLD_2_516_WI_3_validation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
valid.shape

(30490, 29)

In [36]:
#valid prediction value

In [37]:
valid_pred=valid_full[['id', 'd','pred']]
valid_pred=valid_pred.pivot(index="id", columns="d", values="pred").reset_index()

In [38]:
valid_pred.head()

d,id,1886,1887,1888,1889,1890,1891,1892,1893,1894,...,1904,1905,1906,1907,1908,1909,1910,1911,1912,1913
0,FOODS_1_001_CA_1_validation,0.616933,0.486824,0.576643,0.610894,0.775160,0.965206,0.964053,0.724689,0.707894,...,0.876010,1.075088,1.033439,0.748546,0.711271,0.716833,0.697422,0.856404,1.125911,0.980142
1,FOODS_1_001_CA_2_validation,1.054279,1.054811,0.891811,0.989631,1.025219,1.315523,1.009804,0.408544,0.765435,...,0.926265,1.435361,1.551437,0.949698,0.931481,0.912214,1.035248,1.128233,1.248845,1.276809
2,FOODS_1_001_CA_3_validation,0.765058,0.731880,0.754035,0.780718,0.868166,1.245691,1.521489,0.858728,0.818616,...,0.900223,1.353762,1.584653,0.852360,0.766300,0.760927,0.723387,0.781635,1.298789,1.402620
3,FOODS_1_001_CA_4_validation,0.404272,0.347822,0.378831,0.411130,0.466242,0.457052,0.462817,0.454189,0.397169,...,0.369488,0.321209,0.358738,0.394342,0.358976,0.352857,0.382388,0.403064,0.386498,0.386331
4,FOODS_1_001_TX_1_validation,0.414832,0.426224,0.417609,0.415617,0.512621,0.354357,0.466060,0.346745,0.305588,...,0.195761,0.240415,0.211987,0.174900,0.153024,0.071985,0.066141,0.065228,0.081364,0.075794


In [39]:
valid_pred.tail()

d,id,1886,1887,1888,1889,1890,1891,1892,1893,1894,...,1904,1905,1906,1907,1908,1909,1910,1911,1912,1913
30485,HOUSEHOLD_2_516_TX_2_validation,0.201751,0.207788,0.247867,0.245951,0.252856,0.346504,0.323617,0.190287,0.189508,...,0.287342,0.429598,0.376501,0.210071,0.213476,0.232624,0.237850,0.346653,0.406101,0.347502
30486,HOUSEHOLD_2_516_TX_3_validation,0.133671,0.148513,0.199133,0.221699,0.255962,0.283950,0.263446,0.185923,0.180282,...,0.169441,0.219950,0.194314,0.102214,0.118381,0.133748,0.138461,0.160453,0.170231,0.146772
30487,HOUSEHOLD_2_516_WI_1_validation,0.104639,0.090625,0.099793,0.099751,0.116224,0.111738,0.129972,0.104008,0.092932,...,0.098224,0.087717,0.095058,0.061743,0.056050,0.065424,0.064520,0.085878,0.122838,0.115203
30488,HOUSEHOLD_2_516_WI_2_validation,0.050586,0.048221,0.046820,0.048143,0.072698,0.077013,0.060995,0.060295,0.053500,...,0.045603,0.049149,0.047273,0.031855,0.036303,0.036772,0.048712,0.080601,0.088792,0.081109
30489,HOUSEHOLD_2_516_WI_3_validation,0.108540,0.117181,0.106575,0.126703,0.154799,0.160489,0.168201,0.137991,0.127420,...,0.094811,0.121744,0.110320,0.079068,0.066443,0.067206,0.089327,0.098934,0.114462,0.108699


In [40]:
#calculate wrmsse
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from scipy.sparse import csr_matrix
import gc

In [41]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns: #columns
        col_type = df[col].dtypes
        if col_type in numerics: #numerics
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [42]:
# Sales quantities:
sales = pd.read_csv('sales_train_validation.csv')

# Calendar to get week number to join sell prices:
calendar = pd.read_csv('calendar.csv')
calendar = reduce_mem_usage(calendar)

# Sell prices to calculate sales in USD:
sell_prices = pd.read_csv('sell_prices.csv')
sell_prices = reduce_mem_usage(sell_prices)

Mem. usage decreased to  0.12 Mb (41.9% reduction)
Mem. usage decreased to 130.48 Mb (37.5% reduction)


In [43]:
# Dataframe with only last 28 days:
cols = ["d_{}".format(i) for i in range(1914-28, 1914)]
data = sales[["id", 'store_id', 'item_id'] + cols]

# To long form:
data = data.melt(id_vars=["id", 'store_id', 'item_id'], 
                 var_name="d", value_name="sale")

# Add week of year column from 'calendar':
data = pd.merge(data, calendar, how = 'left', 
                left_on = ['d'], right_on = ['d'])

data = data[["id", 'store_id', 'item_id', "sale", "d", "wm_yr_wk"]]

# Add weekly price from 'sell_prices':
data = data.merge(sell_prices, on = ['store_id', 'item_id', 'wm_yr_wk'], how = 'left')
data.drop(columns = ['wm_yr_wk'], inplace=True)

# Calculate daily sales in USD:
data['sale_usd'] = data['sale'] * data['sell_price']
data.head()

,id,store_id,item_id,sale,d,sell_price,sale_usd
0,HOBBIES_1_001_CA_1_validation,CA_1,HOBBIES_1_001,1,d_1886,8.257812,8.257812
1,HOBBIES_1_002_CA_1_validation,CA_1,HOBBIES_1_002,1,d_1886,3.970703,3.970703
2,HOBBIES_1_003_CA_1_validation,CA_1,HOBBIES_1_003,0,d_1886,2.970703,0.000000
3,HOBBIES_1_004_CA_1_validation,CA_1,HOBBIES_1_004,0,d_1886,4.640625,0.000000
4,HOBBIES_1_005_CA_1_validation,CA_1,HOBBIES_1_005,1,d_1886,2.880859,2.880859


In [44]:
# List of categories combinations for aggregations as defined in docs:
dummies_list = [sales.state_id, sales.store_id, 
                sales.cat_id, sales.dept_id, 
                sales.state_id +'_'+ sales.cat_id, sales.state_id +'_'+ sales.dept_id,
                sales.store_id +'_'+ sales.cat_id, sales.store_id +'_'+ sales.dept_id, 
                sales.item_id, sales.state_id +'_'+ sales.item_id, sales.id]

In [45]:
## First element Level_0 aggregation 'all_sales':
dummies_df_list =[pd.DataFrame(np.ones(sales.shape[0]).astype(np.int8), 
                               index=sales.index, columns=['all']).T]

In [46]:
# List of dummy dataframes:
for i, cats in enumerate(dummies_list):
    dummies_df_list +=[pd.get_dummies(cats, drop_first=False, dtype=np.int8).T]

In [47]:
# Concat dummy dataframes in one go:
## Level is constructed for free.
roll_mat_df = pd.concat(dummies_df_list, keys=list(range(12)), 
                        names=['level','id'])#.astype(np.int8, copy=False)

In [48]:
roll_mat_df.head()

0      1      2      3      4      5      6      7      8      \
level id                                                                    
0     all       1      1      1      1      1      1      1      1      1   
1     CA        1      1      1      1      1      1      1      1      1   
      TX        0      0      0      0      0      0      0      0      0   
      WI        0      0      0      0      0      0      0      0      0   
2     CA_1      1      1      1      1      1      1      1      1      1   

            9      ...  30480  30481  30482  30483  30484  30485  30486  \
level id           ...                                                    
0     all       1  ...      1      1      1      1      1      1      1   
1     CA        1  ...      0      0      0      0      0      0      0   
      TX        0  ...      0      0      0      0      0      0      0   
      WI        0  ...      1      1      1      1      1      1      1   
2     CA_1      1  ...      0      0      0      0      0      0      0   

            30487  30488  30489  
level id                         
0     all       1      1      1  
1     CA        0      0      0  
      TX        0      0      0  
      WI        1      1      1  
2     CA_1      0      0      0  

[5 rows x 30490 columns]

In [49]:
roll_mat_df.tail()

0      1      2      3      4      \
level id                                                                   
11    HOUSEHOLD_2_516_TX_2_validation      0      0      0      0      0   
      HOUSEHOLD_2_516_TX_3_validation      0      0      0      0      0   
      HOUSEHOLD_2_516_WI_1_validation      0      0      0      0      0   
      HOUSEHOLD_2_516_WI_2_validation      0      0      0      0      0   
      HOUSEHOLD_2_516_WI_3_validation      0      0      0      0      0   

                                       5      6      7      8      9      ...  \
level id                                                                  ...   
11    HOUSEHOLD_2_516_TX_2_validation      0      0      0      0      0  ...   
      HOUSEHOLD_2_516_TX_3_validation      0      0      0      0      0  ...   
      HOUSEHOLD_2_516_WI_1_validation      0      0      0      0      0  ...   
      HOUSEHOLD_2_516_WI_2_validation      0      0      0      0      0  ...   
      HOUSEHOLD_2_516_WI_3_validation      0      0      0      0      0  ...   

                                       30480  30481  30482  30483  30484  \
level id                                                                   
11    HOUSEHOLD_2_516_TX_2_validation      0      0      0      0      0   
      HOUSEHOLD_2_516_TX_3_validation      0      0      0      0      0   
      HOUSEHOLD_2_516_WI_1_validation      0      0      0      0      0   
      HOUSEHOLD_2_516_WI_2_validation      0      0      0      0      0   
      HOUSEHOLD_2_516_WI_3_validation      0      0      0      0      0   

                                       30485  30486  30487  30488  30489  
level id                                                                  
11    HOUSEHOLD_2_516_TX_2_validation      0      0      0      0      0  
      HOUSEHOLD_2_516_TX_3_validation      0      0      0      0      0  
      HOUSEHOLD_2_516_WI_1_validation      0      0      0      0      0  
      HOUSEHOLD_2_516_WI_2_validation      0      0      0      0      0  
      HOUSEHOLD_2_516_WI_3_validation      0      0      0      0      0  

[5 rows x 30490 columns]

In [50]:
# Save values as sparse matrix & save index for future reference:
roll_index = roll_mat_df.index
roll_mat_csr = csr_matrix(roll_mat_df.values)
roll_mat_csr.shape

(42840, 30490)

In [51]:
# Dump roll matrix to pickle:
roll_mat_df.to_pickle('roll_mat_df.pkl')

In [52]:
# Free some momory:
del dummies_df_list, roll_mat_df
gc.collect()

203

In [53]:
# Fucntion to calculate S weights:
def get_s(drop_days=0):
    
    """
    drop_days: int, equals 0 by default, so S is calculated on all data.
               If equals 28, last 28 days won't be used in calculating S.
    """
    # Rollup sales:
    d_name = ['d_' + str(i+1) for i in range(1913-drop_days)]
    sales_train_val = roll_mat_csr * sales[d_name].values

    no_sales = np.cumsum(sales_train_val, axis=1) == 0
    sales_train_val = np.where(no_sales, np.nan, sales_train_val)

    # Denominator of RMSSE / RMSSE
    weight1 = np.nanmean(np.diff(sales_train_val,axis=1)**2,axis=1)
    
    return weight1

In [54]:
S = get_s(drop_days=0)

In [55]:
S

array([3.51176267e+07, 7.34126518e+06, 3.33954805e+06, ...,
       1.71293871e-01, 6.98666667e-02, 2.81004710e-01])

In [56]:
# Functinon to calculate weights:
def get_w(sale_usd):
    """
    """
    # Calculate the total sales in USD for each item id:
    total_sales_usd = sale_usd.groupby(
        ['id'], sort=False)['sale_usd'].apply(np.sum).values
    
    # Roll up total sales by ids to higher levels:
    weight2 = roll_mat_csr * total_sales_usd
    
    return 12*weight2/np.sum(weight2)

In [57]:
W = get_w(data[['id','sale_usd']])

In [58]:
W 

array([1.00000000e+00, 4.42369608e-01, 2.69296359e-01, ...,
       1.58512584e-06, 1.58512584e-06, 0.00000000e+00])

In [60]:
# Predicted weights
W_df = pd.DataFrame(W,index = roll_index,columns=['w'])

# Load the original weights from github:
W_original_df = pd.read_csv('weights_validation.csv')

# Set new index, calculate difference between original and predicted:
W_original_df = W_original_df.set_index(W_df.index)
W_original_df['Predicted'] = W_df.w
W_original_df['diff'] = W_original_df.Weight - W_original_df.Predicted

# See where we are off by more than e-6
m = W_original_df.Weight.values - W_df.w.values > 0.000001
W_original_df[m]

Level_id  Agg_Level_1  Agg_Level_2    Weight  Predicted  \
level id                                                                       
1     CA               Level2           CA            X  0.442371   0.442370   
3     HOBBIES          Level4      HOBBIES            X  0.128079   0.128075   
      HOUSEHOLD        Level4    HOUSEHOLD            X  0.303335   0.303330   
4     FOODS_1          Level5      FOODS_1            X  0.062625   0.062623   
      FOODS_2          Level5      FOODS_2            X  0.154642   0.154639   
      HOBBIES_1        Level5    HOBBIES_1            X  0.122088   0.122084   
      HOUSEHOLD_1      Level5  HOUSEHOLD_1            X  0.229594   0.229592   
      HOUSEHOLD_2      Level5  HOUSEHOLD_2            X  0.073741   0.073738   
5     CA_HOBBIES       Level6           CA      HOBBIES  0.058855   0.058852   
      CA_HOUSEHOLD     Level6           CA    HOUSEHOLD  0.142772   0.142769   
      TX_HOUSEHOLD     Level6           TX    HOUSEHOLD  0.086420   0.086419   
      WI_HOBBIES       Level6           WI      HOBBIES  0.027931   0.027930   
6     CA_FOODS_2       Level7           CA      FOODS_2  0.057655   0.057654   
      CA_HOBBIES_1     Level7           CA    HOBBIES_1  0.056463   0.056460   
      CA_HOUSEHOLD_1   Level7           CA  HOUSEHOLD_1  0.104863   0.104862   
      CA_HOUSEHOLD_2   Level7           CA  HOUSEHOLD_2  0.037909   0.037907   
      TX_FOODS_1       Level7           TX      FOODS_1  0.016016   0.016015   
      WI_FOODS_2       Level7           WI      FOODS_2  0.062561   0.062560   
      WI_HOBBIES_1     Level7           WI    HOBBIES_1  0.026375   0.026374   
7     CA_2_HOUSEHOLD   Level8         CA_2    HOUSEHOLD  0.037630   0.037629   
      CA_3_HOUSEHOLD   Level8         CA_3    HOUSEHOLD  0.055870   0.055869   
8     WI_2_FOODS_2     Level9         WI_2      FOODS_2  0.030535   0.030534   
9     FOODS_2_029     Level10  FOODS_2_029            X  0.002862   0.002861   
      FOODS_2_183     Level10  FOODS_2_183            X  0.002679   0.002677   
      FOODS_3_586     Level10  FOODS_3_586            X  0.005074   0.005073   

                          diff  
level id                        
1     CA              0.000002  
3     HOBBIES         0.000004  
      HOUSEHOLD       0.000005  
4     FOODS_1         0.000002  
      FOODS_2         0.000004  
      HOBBIES_1       0.000004  
      HOUSEHOLD_1     0.000002  
      HOUSEHOLD_2     0.000003  
5     CA_HOBBIES      0.000003  
      CA_HOUSEHOLD    0.000004  
      TX_HOUSEHOLD    0.000001  
      WI_HOBBIES      0.000001  
6     CA_FOODS_2      0.000001  
      CA_HOBBIES_1    0.000003  
      CA_HOUSEHOLD_1  0.000002  
      CA_HOUSEHOLD_2  0.000002  
      TX_FOODS_1      0.000001  
      WI_FOODS_2      0.000002  
      WI_HOBBIES_1    0.000001  
7     CA_2_HOUSEHOLD  0.000001  
      CA_3_HOUSEHOLD  0.000001  
8     WI_2_FOODS_2    0.000001  
9     FOODS_2_029     0.000001  
      FOODS_2_183     0.000001  
      FOODS_3_586     0.000001

In [ ]:
#PS: As we see our index matches Level_ids and Agg levels of the original dataset, so the csr_matrix works accurately.

In [61]:
SW = W/np.sqrt(S)

In [62]:
sw_df = pd.DataFrame(np.stack((S, W, SW), axis=-1),index = roll_index,columns=['s','w','sw'])
sw_df.to_pickle('sw_df.pkl')

In [63]:
sw_df.head()

s         w        sw
level id                                    
0     all   3.511763e+07  1.000000  0.000169
1     CA    7.341265e+06  0.442370  0.000163
      TX    3.339548e+06  0.269296  0.000147
      WI    3.765354e+06  0.288334  0.000149
2     CA_1  7.499427e+05  0.110888  0.000128

In [64]:
# Function to do quick rollups:
def rollup(v):
    '''
    v - np.array of size (30490 rows, n day columns)
    v_rolledup - array of size (n, 42840)
    '''
    return roll_mat_csr*v #(v.T*roll_mat_csr.T).T


# Function to calculate WRMSSE:
def wrmsse(preds, y_true, score_only=False, s = S, w = W, sw=SW):
    '''
    preds - Predictions: pd.DataFrame of size (30490 rows, N day columns)
    y_true - True values: pd.DataFrame of size (30490 rows, N day columns)
    sequence_length - np.array of size (42840,)
    sales_weight - sales weights based on last 28 days: np.array (42840,)
    '''
    
    if score_only:
        return np.sum(
                np.sqrt(
                    np.mean(
                        np.square(rollup(preds.values-y_true.values))
                            ,axis=1)) * sw)/12 #<-used to be mistake here
    else: 
        score_matrix = (np.square(rollup(preds.values-y_true.values)) * np.square(w)[:, None])/ s[:, None]
        score = np.sum(np.sqrt(np.mean(score_matrix,axis=1)))/12 #<-used to be mistake here
        return score, score_matrix

In [65]:
# Load S and W weights for WRMSSE calcualtions:
sw_df = pd.read_pickle('sw_df.pkl')
S = sw_df.s.values
W = sw_df.w.values
SW = sw_df.sw.values

# Load roll up matrix to calcualte aggreagates:
roll_mat_df = pd.read_pickle('roll_mat_df.pkl')
roll_index = roll_mat_df.index
roll_mat_csr = csr_matrix(roll_mat_df.values)
del roll_mat_df

In [66]:
valid.drop(['id'], axis=1, inplace=True)

In [67]:
valid.head()

d,1886,1887,1888,1889,1890,1891,1892,1893,1894,1895,...,1904,1905,1906,1907,1908,1909,1910,1911,1912,1913
0,2.0,1.0,1.0,0.0,4.0,0.0,0.0,4.0,1.0,3.0,...,0.0,2.0,0.0,4.0,1.0,1.0,0.0,1.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2.0,...,1.0,0.0,14.0,0.0,1.0,1.0,4.0,0.0,0.0,4.0
2,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,13.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,1.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [68]:
y_true=valid

In [69]:
y_true.head()

d,1886,1887,1888,1889,1890,1891,1892,1893,1894,1895,...,1904,1905,1906,1907,1908,1909,1910,1911,1912,1913
0,2.0,1.0,1.0,0.0,4.0,0.0,0.0,4.0,1.0,3.0,...,0.0,2.0,0.0,4.0,1.0,1.0,0.0,1.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2.0,...,1.0,0.0,14.0,0.0,1.0,1.0,4.0,0.0,0.0,4.0
2,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,13.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,1.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [70]:
valid_pred.drop(['id'], axis=1, inplace=True)

In [71]:
valid_pred.head()

d,1886,1887,1888,1889,1890,1891,1892,1893,1894,1895,...,1904,1905,1906,1907,1908,1909,1910,1911,1912,1913
0,0.616933,0.486824,0.576643,0.610894,0.775160,0.965206,0.964053,0.724689,0.707894,0.712228,...,0.876010,1.075088,1.033439,0.748546,0.711271,0.716833,0.697422,0.856404,1.125911,0.980142
1,1.054279,1.054811,0.891811,0.989631,1.025219,1.315523,1.009804,0.408544,0.765435,0.969218,...,0.926265,1.435361,1.551437,0.949698,0.931481,0.912214,1.035248,1.128233,1.248845,1.276809
2,0.765058,0.731880,0.754035,0.780718,0.868166,1.245691,1.521489,0.858728,0.818616,0.834893,...,0.900223,1.353762,1.584653,0.852360,0.766300,0.760927,0.723387,0.781635,1.298789,1.402620
3,0.404272,0.347822,0.378831,0.411130,0.466242,0.457052,0.462817,0.454189,0.397169,0.427144,...,0.369488,0.321209,0.358738,0.394342,0.358976,0.352857,0.382388,0.403064,0.386498,0.386331
4,0.414832,0.426224,0.417609,0.415617,0.512621,0.354357,0.466060,0.346745,0.305588,0.403812,...,0.195761,0.240415,0.211987,0.174900,0.153024,0.071985,0.066141,0.065228,0.081364,0.075794


In [72]:
score = wrmsse(valid_pred, y_true, score_only=True)
score

0.37599810707606224

In [73]:
score1, score_matrix = wrmsse(valid_pred, y_true)
score_df = pd.DataFrame(score_matrix, index = roll_index)
score_df.reset_index(inplace=True)
score_df.head()

,level,id,0,1,2,3,4,5,6,7,...,18,19,20,21,22,23,24,25,26,27
0,0,all,0.017035,0.001227,0.027967,0.020217,0.000179,0.040143,0.000019,0.002946,...,0.049805,0.088307,0.023908,0.001255,0.180378,0.014461,0.000166,0.010365,9.115981e-03,0.000609
1,1,CA,0.028090,0.000176,0.031749,0.024148,0.009038,0.019507,0.001530,0.002733,...,0.013376,0.033008,0.003968,0.006059,0.040050,0.005560,0.000176,0.002067,4.169869e-03,0.000167
2,1,TX,0.000056,0.000425,0.000969,0.002082,0.003455,0.003835,0.000148,0.000238,...,0.002124,0.001368,0.008526,0.000101,0.016808,0.000796,0.000026,0.003565,6.712752e-04,0.000952
3,1,WI,0.002040,0.000489,0.000271,0.003872,0.001523,0.000173,0.000740,0.000249,...,0.002005,0.003484,0.000204,0.002477,0.003736,0.000092,0.000020,0.000142,6.751893e-07,0.000006
4,2,CA_1,0.004645,0.002215,0.002858,0.002067,0.000045,0.001536,0.000533,0.000027,...,0.000547,0.000786,0.001242,0.000410,0.000007,0.000202,0.000132,0.000391,9.302990e-05,0.000257


In [74]:
score1

0.37599810707606224

In [82]:
# next step:
# Improvement should come from:
# bayesian optimization: store_id:tried and did not work because can not based on wrmsse
# Loss function: wrmse: did not work because could not use full data
# Stable CV: by week-year: some store improved and some store did not
# Good features reduction strategy
# Predictions stabilization with NN